In [1]:
import string
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

#### Prepare for Dataset

In [2]:
all_chars       = string.printable
n_chars         = len(all_chars)
file            = open('./shakespeare.txt').read()
file_len        = len(file)

print('Length of file: {}'.format(file_len))
print('All possible characters: {}'.format(all_chars))
print('Number of all possible characters: {}'.format(n_chars))

Length of file: 1115394
All possible characters: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

Number of all possible characters: 100


In [3]:
# Get a random sequence of the Shakespeare dataset.
def get_random_seq():
    seq_len     = 128  # The length of an input sequence.
    start_index = random.randint(0, file_len - seq_len)
    end_index   = start_index + seq_len + 1
    return file[start_index:end_index]

# Convert the sequence to one-hot tensor.
def seq_to_onehot(seq):
    tensor = torch.zeros(len(seq), 1, n_chars) 
    # Shape of the tensor:
    #     (sequence length, batch size, classes)
    # Here we use batch size = 1 and classes = number of unique characters.
    for t, char in enumerate(seq):
        index = all_chars.index(char)
        tensor[t][0][index] = 1
    return tensor

# Convert the sequence to index tensor.
def seq_to_index(seq):
    tensor = torch.zeros(len(seq), 1)
    # Shape of the tensor: 
    #     (sequence length, batch size).
    # Here we use batch size = 1.
    for t, char in enumerate(seq):
        tensor[t] = all_chars.index(char)
    return tensor

# Sample a mini-batch including input tensor and target tensor.
def get_input_and_target():
    seq    = get_random_seq()
    input  = seq_to_onehot(seq[:-1])      # Input is represented in one-hot.
    target = seq_to_index(seq[1:]).long() # Target is represented in index.
    return input, target

#### Choose a Device

In [4]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  
# If 'cuda:0' is printed, it means GPU is available.

cuda:0


#### Network Definition

In [5]:
class Net(nn.Module):
    def __init__(self):
        # Initialization.
        super(Net, self).__init__()
        self.input_size  = n_chars   # Input size: Number of unique chars.
        self.hidden_size = 1000       # Hidden size: 100.
        self.output_size = n_chars   # Output size: Number of unique chars.
        
        self.func1 = nn.RNNCell(self.input_size, self.hidden_size)
        self.func2 = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, input, hidden):
        """ Forward function.
              input:  One-hot input. It refers to the x_t in homework write-up.
              hidden: Previous hidden state. It refers to the h_{t-1}.
            Returns (output, hidden) where output refers to y_t and 
                     hidden refers to h_t.
        """
        # Forward function.
        hidden = self.func1(input, hidden)
        output = self.func2(hidden)

        return output, hidden

    def init_hidden(self):
        # Initial hidden state.
        # 1 means batch size = 1.
        return torch.zeros(1, self.hidden_size).to(device) 
    
net = Net()     # Create the network instance.
net.to(device)  # Move the network parameters to the specified device.

Net(
  (func1): RNNCell(100, 1000)
  (func2): Linear(in_features=1000, out_features=100, bias=True)
)

#### Training Step and Evaluation Step

In [6]:
# Training step function.
def train_step(net, opt, input, target):
    """ Training step.
        net:    The network instance.
        opt:    The optimizer instance.
        input:  Input tensor.  Shape: [seq_len, 1, n_chars].
        target: Target tensor. Shape: [seq_len, 1].
    """
    seq_len = input.shape[0]    # Get the sequence length of current input.
    hidden = net.init_hidden()  # Initial hidden state.
    net.zero_grad()             # Clear the gradient.
    loss = 0                    # Initial loss.

    for t in range(seq_len):    # For each one in the input sequence.
        output, hidden = net(input[t], hidden)
        loss += loss_func(output, target[t])

    loss.backward()             # Backward. 
    opt.step()                  # Update the weights.

    return loss / seq_len       # Return the average loss w.r.t sequence length.

In [7]:
# Evaluation step function.
def eval_step(net, init_seq='W', predicted_len=100):
    # Initialize the hidden state, input and the predicted sequence.
    hidden        = net.init_hidden()
    init_input    = seq_to_onehot(init_seq).to(device)
    predicted_seq = init_seq

    # Use initial string to "build up" hidden state.
    for t in range(len(init_seq) - 1):
        output, hidden = net(init_input[t], hidden)
        
    # Set current input as the last character of the initial string.
    input = init_input[-1]
    
    # Predict more characters after the initial string.
    for t in range(predicted_len):
        # Get the current output and hidden state.
        output, hidden = net(input, hidden)
        
        # Sample from the output as a multinomial distribution.
        predicted_index = torch.multinomial(output.view(-1).exp(), 1)[0]
        
        # Add predicted character to the sequence and use it as next input.
        predicted_char  = all_chars[predicted_index]
        predicted_seq  += predicted_char
        
        # Use the predicted character to generate the input of next round.
        input = seq_to_onehot(predicted_char)[0].to(device)

    return predicted_seq

#### Training Procedure

In [8]:
# Number of iterations.
iters       = 12300  # Number of training iterations.
print_iters = 100    # Number of iterations for each log printing.

# The loss variables.
all_losses = []
loss_sum   = 0

# Initialize the optimizer and the loss function.
#################change optimizer ###############################
opt       = torch.optim.Adadelta(net.parameters(), lr=0.005) 
#################change optimizer ###############################
loss_func = nn.CrossEntropyLoss()

count = 0
# Training procedure.
for i in range(iters):
    try:
        input, target = get_input_and_target()            # Fetch input and target.
    except: 
        count += 1
        print("Illegal characters:")
        print(count)
        continue
    input, target = input.to(device), target.to(device) # Move to GPU memory.
    loss      = train_step(net, opt, input, target)   # Calculate the loss.
    loss_sum += loss                                  # Accumulate the loss.

    # Print the log.
    if i % print_iters == print_iters - 1:
        print('iter:{}/{} loss:{}'.format(i, iters, loss_sum / print_iters))
        print('generated sequence: {}\n'.format(eval_step(net)))
              
        # Track the loss.
        all_losses.append(loss_sum / print_iters)
        loss_sum = 0

iter:99/12300 loss:3.7592949867248535
generated sequence: Wl@&/;&
e,eeieEIaTh vhersd.osTaru l
f|tlA 
iem-sbddgymd,'  wIaiRut  amhEe hlhaaf.hI
n yN iino roe 
sn

iter:199/12300 loss:3.364504098892212
generated sequence: Wqd+HA t reIrnnar.
el a iae ohooryylhKalawiCmOtn yr Aheadramtaoe holenl. 
ayeOe  tTardtno tyLlu
ikIoy

iter:299/12300 loss:3.3826065063476562
generated sequence: W'g
 eN ,o kewuKt toreg.,nehoi erlk eadiR
a,i,syeuAo  snhi
Tetpbyoacn;T,ihhr  aluto
lltun  .,as v.i,p

iter:399/12300 loss:3.37135910987854
generated sequence: Wl\-"Rcd  i BbaohoOiiniF esoddew yiR tsw lUntoL 
 Fenoss'e 
rdotil  h

onhrete moo ue,i
 rwe S dp uro

iter:499/12300 loss:3.330958127975464
generated sequence: WQ9Ouhr,aapshon hAtudhdm:
ints ad hgawuaqd  !hre  h
o arfheerln giWndt o-altceedowr A
ienom?,lahm
n  

iter:599/12300 loss:3.352985143661499
generated sequence: WTUCfafnmr, re ddcfI r E gt: os!MdeaNyhb ydhhe  ,,rhRme  teshe r ho aev e .rhmto  eWai u dhwtTEaeL i


iter:699/12300 loss:3.327651977539

#### Training Loss Curve

In [ ]:
plt.xlabel('iters')
plt.ylabel('loss')
plt.plot(all_losses)
plt.show()

In [ ]:
all_losses

#### Evaluation: A Sample of Generated Sequence

In [ ]:
print(eval_step(net, predicted_len=600))

In [ ]:
import numpy as np
f = open("tanh_RNN_1000.npy", "w")

In [ ]:
a = [data.item() for data in all_losses]

In [ ]:
a

In [ ]:
np.save("tanh_RNN_1000.npy", a)

In [ ]:
np.load("tanh_RNN_1000.npy")

In [ ]:
plt.xlabel('iters')
plt.ylabel('loss')
plt.plot(np.load("tanh_RNN_1000.npy"))
plt.show()

In [ ]:
## experiment different plotting

In [ ]:
a = np.random.choice(100, 100)
b = np.random.choice(100, 100)
y = np.arange(100) #length of the loss

In [ ]:
import numpy as np
plt.xlabel('iters')
plt.ylabel('loss')
plt.plot(np.load("original_RNN.npy"), label='RNN(Relu-100)')
# plt.plot(np.load("tanh_RNN_tanh.npy"), label='RNN(tanh)')
# plt.plot(np.load("original_RNN_10.npy"), label='RNN(Relu-10)')
plt.legend(loc='best')
plt.show()